[Reference](https://medium.com/python-in-plain-english/python-constructed-event-flow-functions-d21019bb6690)

In [1]:
events = [
    {"monkey": "Sam",   "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "flinging",    "kind": "feces"     },
    {"monkey": "Sam",   "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "hungry",      "kind": "apple"     },
]

def do_monkey_flinging(event):
    print(f"{event['monkey']}:\tdo_monkey_flinging")

def do_monkey_hungry_apple(event):
    print(f"{event['monkey']}:\tdo_monkey_hungry_apple")

def do_monkey_hungry_banana(event):
    print(f"{event['monkey']}:\tdo_monkey_hungry_banana")

def dispatch_event(event):
    if event["state"] == "hungry":
        if event["kind"] == "banana":
            do_monkey_hungry_banana(event)
        if event["kind"] == "apple":
            do_monkey_hungry_apple(event)
    elif event["state"] == "flinging":
        do_monkey_flinging(event)

for event in events:
    dispatch_event(event)

Sam:	do_monkey_hungry_banana
Chet:	do_monkey_flinging
Sam:	do_monkey_flinging
Sam:	do_monkey_hungry_banana
Chet:	do_monkey_hungry_banana
Chet:	do_monkey_flinging
Sam:	do_monkey_flinging
Sam:	do_monkey_hungry_apple


In [2]:
from collections import Counter

events = [
    {"monkey": "Sam",   "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "flinging",    "kind": "feces"     },
    {"monkey": "Sam",   "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "hungry",      "kind": "apple"     },
]

event_counters = {}

def do_monkey_flinging(event):
    print(f"{event['monkey']}:\tdo_monkey_flinging")

def do_monkey_hungry_apple(event):
    print(f"{event['monkey']}:\tdo_monkey_hungry_apple")

def do_monkey_hungry_banana(event):
    if event['_count'] == 1:
        print(f"{event['monkey']}:\tdo_monkey_hungry_banana")
    else:
        print(f"{event['monkey']}:\tdont_do_monkey_hungry_banana")

def dispatch_event(event):

    # Make sure each monkey has their own event counter
    event_counters.setdefault(event["monkey"], Counter())

    event_counter = event_counters[event["monkey"]]

    # Compose an event name based on the state and kind values of the event
    event_name = f"{event['state']}_{event['kind']}"

    event_counter.update([event_name])
    event_count = event_counter[event_name]

    event['_count'] = event_count

    if event["state"] == "hungry":
        if event["kind"] == "banana":
            do_monkey_hungry_banana(event)
        if event["kind"] == "apple":
            do_monkey_hungry_apple(event)
    elif event["state"] == "flinging":
        do_monkey_flinging(event)

for event in events:
    dispatch_event(event)

Sam:	do_monkey_hungry_banana
Chet:	do_monkey_flinging
Sam:	do_monkey_flinging
Sam:	dont_do_monkey_hungry_banana
Chet:	do_monkey_hungry_banana
Chet:	do_monkey_flinging
Sam:	do_monkey_flinging
Sam:	do_monkey_hungry_apple


In [4]:
import sys
from collections import Counter

root = sys.modules[__name__]

events = [
    {"monkey": "Sam",   "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "flinging",    "kind": "feces"     },
    {"monkey": "Sam",   "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "hungry",      "kind": "apple"     },
]

event_counters = {}

def do_monkey_flinging(event):
    print(f"{event['monkey']}:\tdo_monkey_flinging")

def do_monkey_hungry_apple(event):
    print(f"{event['monkey']}:\tdo_monkey_hungry_apple")

def do_monkey_hungry_banana_1(event):
    print(f"{event['monkey']}:\tdo_monkey_hungry_banana_1")

def dispatch_event(event):

    # Make sure each monkey has their own event counter
    event_counters.setdefault(event["monkey"], Counter())

    event_counter = event_counters[event["monkey"]]

    # Compose an event name based on the state and kind values of the event
    event_name = f"{event['state']}_{event['kind']}"

    # Update counter
    event_counter.update([event_name])
    event_count = event_counter[event_name]

    # Pass along a useful tidbit if needed
    event['_count'] = event_count

    callback = None

    callback = getattr(root, f"do_monkey_{event['state']}", callback)
    callback = getattr(root, f"do_monkey_{event['state']}_{event['kind']}", callback)
    callback = getattr(root, f"do_monkey_{event['state']}_{event['kind']}_{event_count}", callback)

    if callback:
        callback(event)

for event in events:
    dispatch_event(event)

Sam:	do_monkey_hungry_banana_1
Chet:	do_monkey_flinging
Sam:	do_monkey_flinging
Sam:	dont_do_monkey_hungry_banana
Chet:	do_monkey_hungry_banana_1
Chet:	do_monkey_flinging
Sam:	do_monkey_flinging
Sam:	do_monkey_hungry_apple


In [5]:
import sys
from collections import Counter

import inflect

root = sys.modules[__name__]

inflector = inflect.engine()

events = [
    {"monkey": "Sam",   "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "hungry",      "kind": "banana"    },
    {"monkey": "Chet",  "state": "flinging",    "kind": "feces"     },
    {"monkey": "Sam",   "state": "flinging",    "kind": "grass"     },
    {"monkey": "Sam",   "state": "hungry",      "kind": "apple"     },
]

event_counters = {}

def do_monkey_flinging(event):
    print(f"{event['monkey']}:\tdo_monkey_flinging")

def do_monkey_hungry_apple(event):
    print(f"{event['monkey']}:\tdo_monkey_hungry_apple")

def do_monkey_hungry_banana_first(event):
    print(f"{event['monkey']}:\tdo_monkey_hungry_banana_first")

def do_monkey_hungry_banana_second(event):
    print(f"{event['monkey']}:\tdo_monkey_hungry_banana_second (oh no!)")

def dispatch_event(event):

    # Make sure each monkey has their own event counter
    event_counters.setdefault(event["monkey"], Counter())

    event_counter = event_counters[event["monkey"]]

    # Compose an event name based on the state and kind values of the event
    event_name = f"{event['state']}_{event['kind']}"

    # Update counter
    event_counter.update([event_name])
    event_count = event_counter[event_name]
    event_count_words = inflector.number_to_words(inflector.ordinal(event_count)).replace(' ', '_')

    # Pass along a useful tidbit if needed
    event['_count'] = event_count

    callback = None

    callback = getattr(root, f"do_monkey_{event['state']}", callback)
    callback = getattr(root, f"do_monkey_{event['state']}_{event['kind']}", callback)
    callback = getattr(root, f"do_monkey_{event['state']}_{event['kind']}_{event_count_words}", callback)

    if callback:
        callback(event)

for event in events:
    dispatch_event(event)

Sam:	do_monkey_hungry_banana_first
Chet:	do_monkey_flinging
Sam:	do_monkey_flinging
Sam:	do_monkey_hungry_banana_second (oh no!)
Chet:	do_monkey_hungry_banana_first
Chet:	do_monkey_flinging
Sam:	do_monkey_flinging
Sam:	do_monkey_hungry_apple
